In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!pip install catboost
!pip install optuna

# 0. Importar bibliotecas

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import time
import lightgbm as lgb
import catboost as cb
import joblib
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
import optuna
from optuna.samplers import TPESampler
from optuna.study import MaxTrialsCallback
from optuna.trial import TrialState

# Suprimir os alertas do LightGBM
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')

# 1. Leitura dos datasets

In [12]:
# Note que neste momento importamos as bases contendo todas as variáveis criadas no dataprep

train = pd.read_csv('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/train_002.csv')
val = pd.read_csv('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/val_002.csv')

In [13]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,oil_price,city,state,type,...,sales_rolling_mean_180,sales_rolling_mean_360,year,month,day,day_of_week,is_weekend,sales_oil_price_interaction_mult,sales_oil_price_interaction_div,target
0,1291950,2014-12-29,1,AUTOMOTIVE,1.0,0,53.46,Quito,Pichincha,D,...,3.066667,2.811111,2014,12,29,0,False,53.46,0.018706,44.0
1,1293732,2014-12-30,1,AUTOMOTIVE,4.0,0,54.14,Quito,Pichincha,D,...,3.083333,2.822222,2014,12,30,1,False,216.56,0.073883,40.0
2,1295514,2014-12-31,1,AUTOMOTIVE,0.0,0,53.45,Quito,Pichincha,D,...,3.061111,2.822222,2014,12,31,2,False,0.00,0.000000,43.0
3,1299078,2015-01-02,1,AUTOMOTIVE,2.0,0,52.72,Quito,Pichincha,D,...,3.022222,2.808333,2015,1,2,4,False,105.44,0.037936,44.0
4,1300860,2015-01-03,1,AUTOMOTIVE,6.0,0,50.05,Quito,Pichincha,D,...,3.050000,2.819444,2015,1,3,5,True,250.25,0.099900,43.0


In [14]:
val.head()

,id,date,store_nbr,family,sales,onpromotion,oil_price,city,state,type,...,sales_rolling_mean_180,sales_rolling_mean_360,year,month,day,day_of_week,is_weekend,sales_oil_price_interaction_mult,sales_oil_price_interaction_div,target
0,2756754,2017-04-01,1,AUTOMOTIVE,9.0,0,50.25,Quito,Pichincha,D,...,3.783333,4.069444,2017,4,1,5,True,552.75,0.218905,79.0
1,2758536,2017-04-02,1,AUTOMOTIVE,4.0,0,50.25,Quito,Pichincha,D,...,3.777778,4.061111,2017,4,2,6,True,552.75,0.218905,76.0
2,2760318,2017-04-03,1,AUTOMOTIVE,11.0,0,50.25,Quito,Pichincha,D,...,3.811111,4.077778,2017,4,3,0,False,552.75,0.218905,66.0
3,2762100,2017-04-04,1,AUTOMOTIVE,3.0,0,50.99,Quito,Pichincha,D,...,3.777778,4.061111,2017,4,4,1,False,152.97,0.058835,66.0
4,2763882,2017-04-05,1,AUTOMOTIVE,5.0,0,51.14,Quito,Pichincha,D,...,3.750000,4.063889,2017,4,5,2,False,255.70,0.097771,65.0


# 2. Dados dos custos

In [15]:
df_custos = pd.read_excel('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/dados_avaliacao_financeira.xlsx')
df_custos

,family,Valor Custo de Oportunidade (unidade monetária),Valor Custo de Estoque (unidade monetária)
0,AUTOMOTIVE,400,60.0
1,BABY CARE,80,8.0
2,BEAUTY,60,7.2
3,BEVERAGES,40,3.2
4,BOOKS,50,2.5
5,BREAD/BAKERY,25,5.0
6,CELEBRATION,120,14.4
7,CLEANING,50,5.0
8,DAIRY,70,10.5
9,DELI,45,5.4


In [16]:
# Definir features e target
features = [col for col in train.columns if col not in ['id','date','target']]
target = 'target'

# Separar as colunas numéricas e categóricas
numeric_features = train[features].select_dtypes(include=['int64', 'float64']).columns
categorical_features = train[features].select_dtypes(include=['object']).columns

train.loc[:,categorical_features] = train[categorical_features].astype(str)
train.loc[:,categorical_features] = train[categorical_features].astype(str)

X_train = train[features]
y_train = train[target]
X_test = val[features]
y_test = val[target]

# 3. Modelos - único e segmentados

Para as lojas 1, 4 e 5 são utilizados modelos segmentados, enquanto para as lojas 2 e 3 aplica-se o modelo único desenvolvido.

## 3.1 Modelo único

In [17]:
# Importando pipeline e realizando as predições no dataset de test
pipeline_best_model_catboost = joblib.load('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/pipeline_best_model_all_features_catboost.pkl')
preds_catboost = pd.DataFrame(pipeline_best_model_catboost.predict(X_test))
preds_catboost.index = X_test.index
yreal = pd.DataFrame(y_test)

# Avaliando a performance desse modelo em prever as demandas de todas as lojas
df = pd.concat([X_test,preds_catboost,yreal],axis=1).rename(columns={0:'pred'}).copy()

# Função para calcular o custo de oportunidade e o custo de estoque
def calcular_custos(df, df_custos):
    # Mesclar os dataframes para trazer os custos específicos por 'family'
    df_merged = df.merge(df_custos, on='family', how='left')

    # Custo de estoque: previsão maior que o valor real
    df_merged['custo_estoque'] = df_merged.apply(lambda row: max(row['pred'] - row['target'], 0) * row['Valor Custo de Estoque (unidade monetária)'], axis=1)

    # Custo de oportunidade: previsão menor que o valor real
    df_merged['custo_oportunidade'] = df_merged.apply(lambda row: max(row['target'] - row['pred'], 0) * row['Valor Custo de Oportunidade (unidade monetária)'], axis=1)

    return df_merged

# Aplicar a função ao DataFrame
df_resultado = calcular_custos(df, df_custos)

custo_total_modelo_unico = df_resultado.custo_estoque.sum() + df_resultado.custo_oportunidade.sum()
formatted_cost = f'R$ {custo_total_modelo_unico:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.')
print(f'Custo total para o modelo único: {formatted_cost}')

Custo total para o modelo único: R$ 73.085.864,41


### 3.1.1 Aplicado à Loja 2

In [18]:
df_loja2 = df_resultado[df_resultado.store_nbr == 2].copy()
y_loja2 = y_test[np.isin(y_test.index,df_loja2.index)].copy()

### 3.1.2 Aplicado à Loja 3

In [19]:
df_loja3 = df_resultado[df_resultado.store_nbr == 3].copy()
y_loja3 = y_test[np.isin(y_test.index,df_loja3.index)].copy()

## 3.2 Modelos segmentados

### 3.2.1 Modelo segmentado - Loja 1

In [20]:
# Importando pipeline e realizando as predições no dataset de test
pipeline_segmentado_loja1 = joblib.load('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/pipeline_modelo_segmentado_loja1.pkl')
preds_segmentado_loja1 = pd.DataFrame(pipeline_segmentado_loja1.predict(X_test[X_test.store_nbr == 1]))
preds_segmentado_loja1.index = X_test[X_test.store_nbr == 1].index
y_loja1 = y_test[np.isin(y_test.index,preds_segmentado_loja1.index)].copy()
yreal = pd.DataFrame(y_loja1)

# Avaliando a performance desse modelo em prever as demandas de todas as lojas
aux = pd.concat([X_test[X_test.store_nbr == 1],preds_segmentado_loja1,yreal],axis=1).rename(columns={0:'pred'}).copy()

df_loja1 = calcular_custos(aux, df_custos)

### 3.2.2 Modelo segmentado - Loja 4

In [21]:
# Importando pipeline e realizando as predições no dataset de test
pipeline_segmentado_loja4 = joblib.load('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/pipeline_modelo_segmentado_loja4.pkl')
preds_segmentado_loja4 = pd.DataFrame(pipeline_segmentado_loja4.predict(X_test[X_test.store_nbr == 4]))
preds_segmentado_loja4.index = X_test[X_test.store_nbr == 4].index
y_loja4 = y_test[np.isin(y_test.index,preds_segmentado_loja4.index)].copy()
yreal = pd.DataFrame(y_loja4)

# Avaliando a performance desse modelo em prever as demandas de todas as lojas
aux = pd.concat([X_test[X_test.store_nbr == 4],preds_segmentado_loja4,yreal],axis=1).rename(columns={0:'pred'}).copy()

df_loja4 = calcular_custos(aux, df_custos)

### 3.2.3 Modelo segmentado - Loja 5

In [22]:
# Importando pipeline e realizando as predições no dataset de test
pipeline_segmentado_loja5 = joblib.load('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/pipeline_modelo_segmentado_loja5.pkl')
preds_segmentado_loja5 = pd.DataFrame(pipeline_segmentado_loja5.predict(X_test[X_test.store_nbr == 5]))
preds_segmentado_loja5.index = X_test[X_test.store_nbr == 5].index
y_loja5 = y_test[np.isin(y_test.index,preds_segmentado_loja5.index)].copy()
yreal = pd.DataFrame(y_loja5)

# Avaliando a performance desse modelo em prever as demandas de todas as lojas
aux = pd.concat([X_test[X_test.store_nbr == 5],preds_segmentado_loja5,yreal],axis=1).rename(columns={0:'pred'}).copy()

df_loja5 = calcular_custos(aux, df_custos)

### 3.2.4 Concatenando todos os datasets (segmentados)

In [23]:
df_segmentado = pd.concat([df_loja1,df_loja2,df_loja3,df_loja4,df_loja5],axis=0)
df_segmentado.head()

,store_nbr,family,sales,onpromotion,oil_price,city,state,type,cluster,transactions,...,day_of_week,is_weekend,sales_oil_price_interaction_mult,sales_oil_price_interaction_div,pred,target,Valor Custo de Oportunidade (unidade monetária),Valor Custo de Estoque (unidade monetária),custo_estoque,custo_oportunidade
0,1,AUTOMOTIVE,9.0,0,50.25,Quito,Pichincha,D,13,1477.0,...,5,True,552.75,0.218905,201.757096,79.0,400,60.0,7365.425752,0.0
1,1,AUTOMOTIVE,4.0,0,50.25,Quito,Pichincha,D,13,557.0,...,6,True,552.75,0.218905,205.239844,76.0,400,60.0,7754.390649,0.0
2,1,AUTOMOTIVE,11.0,0,50.25,Quito,Pichincha,D,13,1759.0,...,0,False,552.75,0.218905,203.195436,66.0,400,60.0,8231.726157,0.0
3,1,AUTOMOTIVE,3.0,0,50.99,Quito,Pichincha,D,13,1747.0,...,1,False,152.97,0.058835,200.492503,66.0,400,60.0,8069.550160,0.0
4,1,AUTOMOTIVE,5.0,0,51.14,Quito,Pichincha,D,13,1930.0,...,2,False,255.70,0.097771,198.390461,65.0,400,60.0,8003.427647,0.0


## 3.3 Modelo vigente

In [24]:
# Carregar o modelo salvo
pipeline_vigente = joblib.load('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/pipeline_model.pkl')

# Definir features e target
features = [col for col in train.columns if col not in ['id','date','target']]
target = 'target'

# Separar as colunas numéricas e categóricas
numeric_features = train[features].select_dtypes(include=['int64', 'float64']).columns
categorical_features = train[features].select_dtypes(include=['object']).columns

train.loc[:,categorical_features] = train[categorical_features].astype(str)
train.loc[:,categorical_features] = train[categorical_features].astype(str)

X_train = train[features]
y_train = train[target]
X_test = val[features]
y_test = val[target]

# Previsões no treino
train_predictions = pipeline_vigente.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
train_mae = mean_absolute_error(y_train, train_predictions)

# Previsões no teste
test_predictions = pipeline_vigente.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
test_mae = mean_absolute_error(y_test, test_predictions)

results = []
model_name = 'Modelo Vigente'

results.append((model_name, train_rmse, test_rmse, train_mae, test_mae))

# Mostrar os resultados
results_df1_vigente = pd.DataFrame(results, columns=['Model', 'Train RMSE', 'Test RMSE', 'Train MAE', 'Test MAE'])

results_df1_vigente

,Model,Train RMSE,Test RMSE,Train MAE,Test MAE
0,Modelo Vigente,10948.499005,10910.384667,3395.166383,3287.100141


In [25]:
aux = pd.concat([X_test,y_test,pd.DataFrame(test_predictions)],axis=1).rename(columns={0:'pred'})
df_vigente = calcular_custos(aux, df_custos)
df_vigente.head()

,store_nbr,family,sales,onpromotion,oil_price,city,state,type,cluster,transactions,...,day_of_week,is_weekend,sales_oil_price_interaction_mult,sales_oil_price_interaction_div,target,pred,Valor Custo de Oportunidade (unidade monetária),Valor Custo de Estoque (unidade monetária),custo_estoque,custo_oportunidade
0,1,AUTOMOTIVE,9.0,0,50.25,Quito,Pichincha,D,13,1477.0,...,5,True,552.75,0.218905,79.0,84.560431,400,60.0,333.625839,0.0
1,1,AUTOMOTIVE,4.0,0,50.25,Quito,Pichincha,D,13,557.0,...,6,True,552.75,0.218905,76.0,84.560431,400,60.0,513.625839,0.0
2,1,AUTOMOTIVE,11.0,0,50.25,Quito,Pichincha,D,13,1759.0,...,0,False,552.75,0.218905,66.0,84.560431,400,60.0,1113.625839,0.0
3,1,AUTOMOTIVE,3.0,0,50.99,Quito,Pichincha,D,13,1747.0,...,1,False,152.97,0.058835,66.0,84.560431,400,60.0,1113.625839,0.0
4,1,AUTOMOTIVE,5.0,0,51.14,Quito,Pichincha,D,13,1930.0,...,2,False,255.70,0.097771,65.0,84.560431,400,60.0,1173.625839,0.0


## 3.4 Comparativo Financeiro

In [26]:
mae_segmentado = np.mean(np.abs(df_segmentado.target - df_segmentado.pred))
mae_unico = np.mean(np.abs(df_resultado.target - df_resultado.pred))
mae_vigente = np.mean(np.abs(df_vigente.target - df_vigente.pred))

print(f'MAE - Modelo vigente: {mae_vigente}')
print(f'MAE - Modelo único: {mae_unico}')
print(f'MAE - Modelo segmentado: {mae_segmentado}\n')

rmse_segmentado = np.sqrt(mean_squared_error(df_segmentado.target, df_segmentado.pred))
rmse_unico = np.sqrt(mean_squared_error(df_resultado.target, df_resultado.pred))
rmse_vigente = np.sqrt(mean_squared_error(df_vigente.target, df_vigente.pred))

print(f'RMSE - Modelo vigente: {rmse_vigente}')
print(f'RMSE - Modelo único: {rmse_unico}')
print(f'RMSE - Modelo segmentado: {rmse_segmentado}\n')

custo_total_modelo_vigente = df_vigente.custo_estoque.sum() + df_vigente.custo_oportunidade.sum()
formatted_cost = f'R$ {custo_total_modelo_vigente:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.')
print(f'Custo total para o modelo vigente: {formatted_cost}')

custo_total_modelo_unico = df_resultado.custo_estoque.sum() + df_resultado.custo_oportunidade.sum()
formatted_cost = f'R$ {custo_total_modelo_unico:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.')
print(f'Custo total para o modelo único: {formatted_cost}')

custo_total_modelo_segmentado = df_segmentado.custo_estoque.sum() + df_segmentado.custo_oportunidade.sum()
formatted_cost = f'R$ {custo_total_modelo_segmentado:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.')
print(f'Custo total para o modelo segmentado: {formatted_cost}\n')


MAE - Modelo vigente: 3287.1001408847296
MAE - Modelo único: 442.2136885003837
MAE - Modelo segmentado: 417.7926691929652

RMSE - Modelo vigente: 10910.384666914948
RMSE - Modelo único: 1004.2182908920599
RMSE - Modelo segmentado: 916.8298885321577

Custo total para o modelo vigente: R$ 893.437.295,27
Custo total para o modelo único: R$ 73.085.864,41
Custo total para o modelo segmentado: R$ 72.220.197,77

